In [7]:
import sys,os
import numpy as np
import pandas as pd
import xarray as xr
import time as tm

import NSSEA as ns
import NSSEA.plot as nsp
import NSSEA.models as nsm
dt_string="06051647"
from cmdstanpy import cmdstan_path, set_cmdstan_path
#This has to be done manually ?
set_cmdstan_path('/home/barbauxo/miniconda3/envs/TestSDFC/bin/cmdstan')
cmdstan_path()

'/home/barbauxo/miniconda3/envs/TestSDFC/bin/cmdstan'

In [8]:
def load_obs(pathInp,type_data):
    ## Load Observations
    dXo = xr.open_dataset(os.path.join( pathInp ,"Observations",type_data,"Xo.nc"))
    Xo  = pd.DataFrame( dXo.tas.values.squeeze() , columns = ["Xo"] , index = dXo.time["time.year"].values )

    Xo #Deja en anomalies
    dYo = xr.open_dataset(os.path.join( pathInp ,"Observations",type_data,"Yo.nc"))
    Yo  = pd.DataFrame( dYo.TX.values.squeeze() , columns = ["Yo"] , index = dYo.time["time.year"].values )
    return Xo,Yo #en celsius

In [9]:
ns_law      = nsm.GEV()

In [10]:
#Prepared data

clim_CX=ns.Climatology.from_netcdf( "clim_CX.nc" , ns_law )

pathInp='/home/barbauxo/Documents/Doctorat/03_Travail/2023_01 Application Tricastin/Data'
assert(os.path.exists(pathInp))
type_data="03_Post_treatment"  #"02_Selected"
Xo,Yo=load_obs(pathInp,type_data)

In [13]:
bayes_kwargs = { "n_ess"   : int(10000/(len(clim_CX.data.sample)-1))   } #Ici produit 10000 tirages en tout
t0 = tm.time()
climCXCB_stan_light=ns.stan_constrain(clim_CX,Yo,'stan_files/GEV_non_stationary.stan', **bayes_kwargs)
t1 = tm.time()
total = t1-t0
print(total)
climCXCB_stan_light.to_netcdf( ("clim_CXCB_stan.nc")  ) 

11:18:23 - cmdstanpy - INFO - CmdStan start processing
11:18:23 - cmdstanpy - INFO - Chain [1] start processing
11:18:23 - cmdstanpy - INFO - Chain [2] start processing
11:18:23 - cmdstanpy - INFO - Chain [3] start processing
11:18:23 - cmdstanpy - INFO - Chain [4] start processing
11:18:23 - cmdstanpy - INFO - Chain [1] done processing
11:18:23 - cmdstanpy - INFO - Chain [3] done processing
11:18:23 - cmdstanpy - INFO - Chain [2] done processing
11:18:23 - cmdstanpy - INFO - Chain [4] done processing
11:18:23 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -4.43462, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -1.01686, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is 1.11148, but must

S00
S01


11:18:23 - cmdstanpy - INFO - CmdStan start processing
11:18:23 - cmdstanpy - INFO - Chain [1] start processing
11:18:23 - cmdstanpy - INFO - Chain [2] start processing
11:18:23 - cmdstanpy - INFO - Chain [3] start processing
11:18:23 - cmdstanpy - INFO - Chain [4] start processing
11:18:23 - cmdstanpy - INFO - Chain [1] done processing
11:18:23 - cmdstanpy - INFO - Chain [4] done processing
11:18:23 - cmdstanpy - INFO - Chain [2] done processing
11:18:23 - cmdstanpy - INFO - Chain [3] done processing
11:18:23 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -5.32881, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -1.25956, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -1.19816, but mus

S02


11:18:23 - cmdstanpy - INFO - Chain [3] done processing
11:18:23 - cmdstanpy - INFO - Chain [4] done processing
11:18:23 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -1.35066, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -1.22914, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -3.30187, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is 1.28164, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is 1.378, but must be less than or equal to 1.000000 (in 'GEV_non_

...

S07


11:18:24 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is 20.7033, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -1.06831, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is 4.17504, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is 4.45414, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -2.81342, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -4.83644,

S08


11:18:25 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -1.60631, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is 3.58463, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is 3.03733, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is 1.31483, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -1.84717, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -1.88928,

S09
2.371900796890259


11:21:11 - cmdstanpy - INFO - CmdStan start processing
11:21:11 - cmdstanpy - INFO - Chain [1] start processing
11:21:11 - cmdstanpy - INFO - Chain [2] start processing
11:21:11 - cmdstanpy - INFO - Chain [3] start processing
11:21:11 - cmdstanpy - INFO - Chain [4] start processing
11:21:11 - cmdstanpy - INFO - Chain [1] done processing
11:21:11 - cmdstanpy - INFO - Chain [4] done processing


S00


11:21:11 - cmdstanpy - INFO - Chain [2] done processing
11:21:11 - cmdstanpy - INFO - Chain [3] done processing
11:21:11 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -12.84, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is 2.59021, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -2.91177, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -1.01055, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is 1.05631, but must be less than or equal to 1.000000 (in 'GEV_non_

S01


11:21:12 - cmdstanpy - INFO - Chain [1] done processing
11:21:12 - cmdstanpy - INFO - Chain [2] done processing
11:21:12 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is 3.26457, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -1.93516, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is 1.04318, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -9.92392, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -3.6987, but must be greater than or equal to -1.000000 (in 'GEV_non_

...

S08


11:21:14 - cmdstanpy - INFO - Chain [1] done processing
11:21:14 - cmdstanpy - INFO - Chain [2] done processing
11:21:14 - cmdstanpy - INFO - Chain [4] done processing
11:21:14 - cmdstanpy - INFO - Chain [3] done processing
11:21:14 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is 9.88074, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is 1.52804, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is 1.9568, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -2.91058, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
Exception: tmpnbrn9e

S09


11:21:14 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is 1.1057, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is 1.71251, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -5.28779, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -1.55637, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -5.44946, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpnbrn9e_8_model_namespace::log_prob: xi is -1.70

3.0315401554107666


11

In [24]:
#Pour comparer, contrainte originale (ici produit 10 tirages)
t0 = tm.time()
bayes_kwargs = { "n_mcmc_drawn_min" :  5000 , "n_mcmc_drawn_max" : 10000 }
bayes_kwargs["transition_type"]="Fixed"
bayes_kwargs["fixed_cov"]=np.array([0.1,0.1,0.1,0.1,0.1])
climCXCB   = ns.constrain_law( clim_CX , Yo , verbose = False , **bayes_kwargs )
t1 = tm.time()
total = t1-t0
print(total)
climCXCB.to_netcdf( ("clim_CXCB.nc")  ) 

29.220379114151
